In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 6.6 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 11.1 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/JU PROJ/ExecutiveTweetsTesla.csv')
df

,Date,Tweet
0,2018-08-07 18:25:02+00:00,Looks like nothing's changed in FinTwit land t...
1,2018-07-05 22:03:47+00:00,"If $TSLA's BOD is indeed populated by living, ..."
2,2018-05-02 22:32:48+00:00,So picked it up late bit is this an earnings r...
3,2018-03-08 23:59:37+00:00,Chief Accounting Officer packs his bags and ru...
4,2017-11-01 23:31:22+00:00,"Believe it or not, this was the 1st $TSLA CC I..."
...,...,...
2612,2022-02-24 13:37:30+00:00,Patiently waiting for @elonmusk to tweet somet...
2613,2021-11-14 23:40:05+00:00,Who do you side with ? $tsla
2614,2021-10-08 11:33:01+00:00,Will Elon’s pivot to Texas help Tesla sharehol...
2615,2017-02-01 14:20:35+00:00,"Worth about 10% up, no? JUST IN: Tesla Motors ..."


In [6]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

,Date,Tweet
0,2018-08-07,Looks like nothing's changed in FinTwit land t...
1,2018-07-05,"If $TSLA's BOD is indeed populated by living, ..."
2,2018-05-02,So picked it up late bit is this an earnings r...
3,2018-03-08,Chief Accounting Officer packs his bags and ru...
4,2017-11-01,"Believe it or not, this was the 1st $TSLA CC I..."
...,...,...
2612,2022-02-24,Patiently waiting for @elonmusk to tweet somet...
2613,2021-11-14,Who do you side with ? $tsla
2614,2021-10-08,Will Elon’s pivot to Texas help Tesla sharehol...
2615,2017-02-01,"Worth about 10% up, no? JUST IN: Tesla Motors ..."


In [7]:
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'])
#   return df[['Tweets']].groupby(df['Date'].dt.date).count()
    tweet = df['Tweet'].groupby(df['Date'].dt.date).count() 
    d1 = pd.DataFrame(tweet)
    return d1
  # if you want output to be `Series`

df1 = tweets_per_day(df)
df1 = df1.reset_index(level=0)
df1

,Date,Tweet
0,2017-01-02,1
1,2017-01-03,2
2,2017-01-04,6
3,2017-01-05,2
4,2017-01-17,1
...,...,...
858,2022-04-25,2
859,2022-04-26,7
860,2022-04-28,5
861,2022-04-29,3


In [8]:
#df = df.groupby(['Date'])['Tweet'].apply(lambda x: '. '.join(x.astype(str))).reset_index()
df

,Date,Tweet
0,2018-08-07,Looks like nothing's changed in FinTwit land t...
1,2018-07-05,"If $TSLA's BOD is indeed populated by living, ..."
2,2018-05-02,So picked it up late bit is this an earnings r...
3,2018-03-08,Chief Accounting Officer packs his bags and ru...
4,2017-11-01,"Believe it or not, this was the 1st $TSLA CC I..."
...,...,...
2612,2022-02-24,Patiently waiting for @elonmusk to tweet somet...
2613,2021-11-14,Who do you side with ? $tsla
2614,2021-10-08,Will Elon’s pivot to Texas help Tesla sharehol...
2615,2017-02-01,"Worth about 10% up, no? JUST IN: Tesla Motors ..."


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [10]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import math

stop_words = stopwords.words('english')
stop_words.remove("not")
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

df.Tweets = df.Tweet.apply(lambda x: preprocess(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Date,Tweet
0,2018-08-07,Looks like nothing's changed in FinTwit land t...
1,2018-07-05,"If $TSLA's BOD is indeed populated by living, ..."
2,2018-05-02,So picked it up late bit is this an earnings r...
3,2018-03-08,Chief Accounting Officer packs his bags and ru...
4,2017-11-01,"Believe it or not, this was the 1st $TSLA CC I..."
...,...,...
2612,2022-02-24,Patiently waiting for @elonmusk to tweet somet...
2613,2021-11-14,Who do you side with ? $tsla
2614,2021-10-08,Will Elon’s pivot to Texas help Tesla sharehol...
2615,2017-02-01,"Worth about 10% up, no? JUST IN: Tesla Motors ..."


In [11]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [12]:
neg = []
neu = []
pos = []
i = 0
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

for tweet in df.Tweet:
  i = i+1
  encoded_tweet = tokenizer(tweet,padding=True, truncation=True,max_length=250, add_special_tokens = True, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  normScores = softmax(scores)  
  neg.append(normScores[0])
  neu.append(normScores[1])
  pos.append(normScores[2])
  print(i,"--> ",normScores)

1 -->  [0.05345358 0.01448532 0.9320609 ]
2 -->  [0.03058946 0.2500392  0.7193712 ]
3 -->  [0.02015938 0.078013   0.9018275 ]
4 -->  [0.0865296  0.11267897 0.80079156]
5 -->  [0.05074809 0.12861106 0.82064086]
6 -->  [0.02773311 0.04826546 0.9240014 ]
7 -->  [0.1107724  0.04857222 0.8406553 ]
8 -->  [0.04243182 0.10131437 0.8562538 ]
9 -->  [0.06150908 0.04209631 0.89639467]
10 -->  [0.08333766 0.01212356 0.90453875]
11 -->  [0.06040803 0.0159123  0.92367953]
12 -->  [0.04105674 0.20987098 0.74907213]
13 -->  [0.02068351 0.3466438  0.63267255]
14 -->  [0.06198201 0.02621281 0.9118053 ]
15 -->  [0.05526634 0.2925684  0.65216535]
16 -->  [0.41440177 0.02293283 0.5626655 ]
17 -->  [0.05270952 0.03382235 0.9134682 ]
18 -->  [0.0272763 0.0244245 0.9482991]
19 -->  [0.0719183  0.02026843 0.90781325]
20 -->  [0.07531182 0.02006318 0.904625  ]
21 -->  [0.11016527 0.02604123 0.86379355]
22 -->  [0.02411403 0.26656318 0.7093227 ]
23 -->  [0.04259983 0.08765236 0.86974776]
24 -->  [0.04315767 0.2

In [13]:
df['negative'] = neg
df['neutral'] = neu
df['positive'] = pos
df

,Date,Tweet,negative,neutral,positive
0,2018-08-07,Looks like nothing's changed in FinTwit land t...,0.053454,0.014485,0.932061
1,2018-07-05,"If $TSLA's BOD is indeed populated by living, ...",0.030589,0.250039,0.719371
2,2018-05-02,So picked it up late bit is this an earnings r...,0.020159,0.078013,0.901828
3,2018-03-08,Chief Accounting Officer packs his bags and ru...,0.086530,0.112679,0.800792
4,2017-11-01,"Believe it or not, this was the 1st $TSLA CC I...",0.050748,0.128611,0.820641
...,...,...,...,...,...
2612,2022-02-24,Patiently waiting for @elonmusk to tweet somet...,0.061308,0.027580,0.911112
2613,2021-11-14,Who do you side with ? $tsla,0.031450,0.035322,0.933228
2614,2021-10-08,Will Elon’s pivot to Texas help Tesla sharehol...,0.156747,0.009985,0.833268
2615,2017-02-01,"Worth about 10% up, no? JUST IN: Tesla Motors ...",0.201488,0.018120,0.780392


In [14]:
dfx = pd.DataFrame()
po = []
nu = []
ne = []
for i in range (0,863):

  d = df['Date'][i]

  x = df[df['Date'] == d]
  po.append(x['positive'].mean())
  nu.append(x['neutral'].mean())
  ne.append(x['negative'].mean())
dfx['Date'] = df1['Date']
dfx['positive'] = po
dfx['negative'] = ne
dfx['neutral'] = nu
dfx

,Date,positive,negative,neutral
0,2017-01-02,0.658346,0.105337,0.236316
1,2017-01-03,0.635420,0.157270,0.207310
2,2017-01-04,0.632780,0.203407,0.163813
3,2017-01-05,0.800792,0.086530,0.112679
4,2017-01-17,0.678611,0.201350,0.120039
...,...,...,...,...
858,2022-04-25,0.658504,0.116627,0.224869
859,2022-04-26,0.658504,0.116627,0.224869
860,2022-04-28,0.658504,0.116627,0.224869
861,2022-04-29,0.669008,0.105465,0.225527


In [15]:
# Set date as Index
dfx = dfx.set_index('Date')
dfx = dfx.sort_index()
dfx

,positive,negative,neutral
Date,,,
2017-01-02,0.658346,0.105337,0.236316
2017-01-03,0.635420,0.157270,0.207310
2017-01-04,0.632780,0.203407,0.163813
2017-01-05,0.800792,0.086530,0.112679
2017-01-17,0.678611,0.201350,0.120039
...,...,...,...
2022-04-25,0.658504,0.116627,0.224869
2022-04-26,0.658504,0.116627,0.224869
2022-04-28,0.658504,0.116627,0.224869


### STOCK DATA

In [16]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stock_price = pdr.get_data_yahoo("TSLA", start = '2017-01-01', end = '2022-05-07')
print(len(stock_price))
stock_price

[*********************100%***********************]  1 of 1 completed
1346


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,42.972000,44.066002,42.192001,43.397999,43.397999,29616500
2017-01-04,42.950001,45.599998,42.862000,45.397999,45.397999,56067500
2017-01-05,45.284000,45.495998,44.389999,45.349998,45.349998,29558500
2017-01-06,45.386002,46.062000,45.090000,45.801998,45.801998,27639500
2017-01-09,45.793999,46.383999,45.599998,46.256001,46.256001,19897500
...,...,...,...,...,...,...
2022-05-02,860.770020,906.359985,848.030029,902.940002,902.940002,25260500
2022-05-03,903.179993,924.080017,888.590027,909.250000,909.250000,21236500
2022-05-04,903.940002,955.500000,885.280029,952.619995,952.619995,27214600


In [17]:
stock_price = stock_price.drop_duplicates()

# dropping the zero values from volume
stock_price = stock_price[stock_price['Volume'] != 0]

# filtering the important columns required
stock_price = stock_price.filter(['Date', 'Close', 'Open', 'High', 'Low', 'Volume'])

# sorting the data according to the index i.e 'Date'
stock_price = stock_price.sort_index(ascending=True, axis=0)
stock_price

,Close,Open,High,Low,Volume
Date,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600


### COMBINE DATA

In [18]:
stock_data = pd.merge(stock_price,dfx, how='left', left_index=True, right_index=True)

stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,0.635420,0.157270,0.207310
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,0.632780,0.203407,0.163813
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,0.800792,0.086530,0.112679
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,NaN,NaN,NaN
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,NaN,NaN,NaN
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [19]:
stock_data.sort_index(axis = 1)
stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,43.397999,42.972000,44.066002,42.192001,29616500,0.635420,0.157270,0.207310
2017-01-04,45.397999,42.950001,45.599998,42.862000,56067500,0.632780,0.203407,0.163813
2017-01-05,45.349998,45.284000,45.495998,44.389999,29558500,0.800792,0.086530,0.112679
2017-01-06,45.801998,45.386002,46.062000,45.090000,27639500,NaN,NaN,NaN
2017-01-09,46.256001,45.793999,46.383999,45.599998,19897500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,860.770020,906.359985,848.030029,25260500,NaN,NaN,NaN
2022-05-03,909.250000,903.179993,924.080017,888.590027,21236500,NaN,NaN,NaN
2022-05-04,952.619995,903.940002,955.500000,885.280029,27214600,NaN,NaN,NaN


In [20]:
stock_data.isna().sum()

Close         0
Open          0
High          0
Low           0
Volume        0
positive    577
negative    577
neutral     577
dtype: int64

In [21]:
#stock_data.drop(['Tweet'], inplace=True, axis=1)

# rearranging the columns of the whole stock_data
stock_data = stock_data[['Close', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

# displaying the final stock_data
stock_data

,Close,negative,neutral,positive,Open,High,Low,Volume
Date,,,,,,,,
2017-01-03,43.397999,0.157270,0.207310,0.635420,42.972000,44.066002,42.192001,29616500
2017-01-04,45.397999,0.203407,0.163813,0.632780,42.950001,45.599998,42.862000,56067500
2017-01-05,45.349998,0.086530,0.112679,0.800792,45.284000,45.495998,44.389999,29558500
2017-01-06,45.801998,NaN,NaN,NaN,45.386002,46.062000,45.090000,27639500
2017-01-09,46.256001,NaN,NaN,NaN,45.793999,46.383999,45.599998,19897500
...,...,...,...,...,...,...,...,...
2022-05-02,902.940002,NaN,NaN,NaN,860.770020,906.359985,848.030029,25260500
2022-05-03,909.250000,NaN,NaN,NaN,903.179993,924.080017,888.590027,21236500
2022-05-04,952.619995,NaN,NaN,NaN,903.940002,955.500000,885.280029,27214600


In [22]:
stock_data.to_csv('/content/drive/MyDrive/JU PROJ/TSLA FinBERT/EXEC_FINBERT_Tesla_stock_data.csv')